In [3]:
import numpy as np
from scipy.interpolate import griddata
import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS


In [4]:
def read_pts_off(file):
    
        """Read points from an OFF file."""
    
        f = open(file, 'r')
        lines = f.readlines()
        pts = []
        for line in lines[2:]:
            if len(line.split()[:]) == 3:
                pts.append(list(np.float64(line.split()[:])))
        return pts

In [14]:
lst = [[5, 3, 8], [2, 7, 1], [9, 4, 6]]

transposed_lst = list(zip(*lst))

smallest_in_each_col = [min(col) for col in transposed_lst]

print(transposed_lst)  # Output: [2, 3, 1]

[(5, 2, 9), (3, 7, 4), (8, 1, 6)]


In [10]:
print([min(lst, key=lambda x: x[0]), min(lst, key=lambda x: x[1])])

[[2, 7, 1], [5, 3, 8]]


In [ ]:
def read_tiff(file):
    z = rasterio.open(file).read(1)

In [18]:
def out_put_raster(pts, output_raster, cell_size):
    
    """Output a raster file from a list of points."""
    
    # Get the extent of the points
    cell_size = 0.5
    transposed_lst = list(zip(*pts))
    x_range = np.arange(min(transposed_lst[0]), max(transposed_lst[0]) + cell_size, cell_size)
    y_range = np.arange(min(transposed_lst[1]), max(transposed_lst[1]) + cell_size, cell_size)
    grid_x, grid_y = np.meshgrid(x_range, y_range)
    points = [lst[:-1] for lst in pts]
    z = transposed_lst[2]
    grid_z = griddata(points, z, (grid_x, grid_y), method='cubic')
    # Set negative values to 0
    grid_z[grid_z < 0] = 0
    # Define transformation and CRS
    transform = Affine.translation(grid_x[0][0]-cell_size/2, grid_y[0][0]-cell_size/2) * Affine.scale(cell_size, cell_size)
    crs = CRS.from_epsg(2056)

    # Write interpolated raster to file
    interp_raster = rasterio.open(output_raster,
                                'w',
                                driver='GTiff',
                                height=grid_z.shape[0],
                                width=grid_z.shape[1],
                                count=1,
                                dtype=grid_z.dtype,
                                crs=crs,
                                transform=transform)
    interp_raster.write(grid_z, 1)
    interp_raster.close()

In [36]:
pts = read_pts_off('edge_collapse_data/3_0.off')
out_put_raster(pts, 'edge_collapse_data/3_0.tif', 0.5)

In [38]:
pts_2 = read_pts_off("edge_collapse_data/5_0.off")
out_put_raster(pts_2, 'edge_collapse_data/5_0.tif', 2.0)